In [1]:
import pyterrier as pt
if not pt.started():
    pt.init()
from pyterrier_pisa import PisaIndex
import pandas as pd
import pyt_splade
splade = pyt_splade.SpladeFactory()

PyTerrier 0.9.2 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
/home/tiagoalmeida/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
splade.tokenizer.vocab_size

30522

In [ ]:
index = PisaIndex('./msmarco-passage-splade-pisa', stemmer='none')

In [3]:
index.num_docs(), index.num_terms()

(8841823, 28678)

In [6]:
out_df = pd.read_csv("msmarco-passage-splade-pisa/questions_tokinezed")

In [14]:
retr_pipeline = splade.query() >> index.quantized()

In [34]:
out = retr_pipeline.search("define covid")

out["query"][0].split(" ")

['#combine:0=310.09364128112793(#base64(IyN2aWQ=))',
 '#combine:0=249.07488822937012(co)',
 '#combine:0=140.75511693954468(definition)',
 '#combine:0=79.1648268699646(biology)',
 '#combine:0=76.92760229110718(disorder)',
 '#combine:0=76.83706283569336(abbreviation)',
 '#combine:0=61.07416749000549(technology)',
 '#combine:0=59.25496220588684(meaning)',
 '#combine:0=55.682170391082764(anger)',
 '#combine:0=52.32481360435486(brian)',
 '#combine:0=52.32481360435486(davis)',
 '#combine:0=51.043492555618286(harry)',
 '#combine:0=50.57345628738403(ian)',
 '#combine:0=49.864232540130615(jacob)',
 '#combine:0=48.37033152580261(alex)',
 '#combine:0=48.31010699272156(fear)',
 '#combine:0=47.2198486328125(disease)',
 '#combine:0=46.9759464263916(griffin)',
 '#combine:0=46.73144817352295(marcus)',
 '#combine:0=44.87839937210083(henry)',
 '#combine:0=44.25300657749176(morgan)',
 '#combine:0=43.49733293056488(#base64(Ig==))',
 '#combine:0=43.434104323387146(carbon)',
 '#combine:0=42.86321699619293(g

In [35]:
out["query"][0]


'#combine:0=310.09364128112793(#base64(IyN2aWQ=)) #combine:0=249.07488822937012(co) #combine:0=140.75511693954468(definition) #combine:0=79.1648268699646(biology) #combine:0=76.92760229110718(disorder) #combine:0=76.83706283569336(abbreviation) #combine:0=61.07416749000549(technology) #combine:0=59.25496220588684(meaning) #combine:0=55.682170391082764(anger) #combine:0=52.32481360435486(brian) #combine:0=52.32481360435486(davis) #combine:0=51.043492555618286(harry) #combine:0=50.57345628738403(ian) #combine:0=49.864232540130615(jacob) #combine:0=48.37033152580261(alex) #combine:0=48.31010699272156(fear) #combine:0=47.2198486328125(disease) #combine:0=46.9759464263916(griffin) #combine:0=46.73144817352295(marcus) #combine:0=44.87839937210083(henry) #combine:0=44.25300657749176(morgan) #combine:0=43.49733293056488(#base64(Ig==)) #combine:0=43.434104323387146(carbon) #combine:0=42.86321699619293(government) #combine:0=42.48080551624298(culture) #combine:0=41.7115718126297(communication) #

In [38]:

import base64

# Decoding the Base64 encoded string
encoded_str = "#base64(IyN2aWQ=)"
decoded_bytes = base64.b64decode(encoded_str.split("#base64(")[1][:-1])
decoded_str = decoded_bytes.decode('utf-8')
decoded_str

'##vid'

In [16]:
dataset = pt.get_dataset('irds:msmarco-passage')

In [23]:
for doc in dataset.get_corpus_iter():
    if doc["docno"] == "747695":
        print(doc)
        break

msmarco-passage documents:   8%|▊         | 747695/8841823 [00:02<00:27, 292384.97it/s]

{'text': '0º Combiners/Dividers. To Consider When Specifying Your Combiner Isolating Termination Dissipation Isolating terminations enable the dissipation of power due to various unbalances and possible input failures.', 'docno': '747695'}


In [12]:
import json
from collections import defaultdict
import pandas as pd

qrels = defaultdict(dict)
run = {}
    
with open(f"beir_datasets/msmarco/relevant_pairs.jsonl") as f:
    for i, q_data in enumerate(map(json.loads, f)):
        run[q_data["id"]] = q_data["question"]
        qrels[q_data["id"]][q_data["doc_id"]] = 1

        
question_ids, questions = list(zip(*run.items()))

questions_dataframe = pd.DataFrame([{"qid":i, "query":question.lower()} for i,question in enumerate(questions)])

In [8]:
dfs = []
for out_df in splade.query().transform_gen(questions_dataframe, batch_size=16):
    dfs.append(out_df)

In [11]:
out_df = pd.concat(dfs)

In [12]:
out_df.to_csv("msmarco-passage-splade-pisa/questions_tokinezed")

,Unnamed: 0,qid,query_0,query
0,0,0,how many years did william bradford serve as g...,#combine:0=224.13744926452637(bradford) #combi...
1,1,1,define preventive,#combine:0=271.3244915008545(prevent) #combine...
2,2,2,color overlay photoshop,#combine:0=264.2400026321411(color) #combine:0...
3,3,3,____________________ is considered the father ...,#combine:0=243.89445781707764(modern) #combine...
4,4,4,does xpress bet charge to deposit money in you...,#combine:0=246.45178318023682(xp) #combine:0=2...
...,...,...,...,...
6975,6975,6975,difference between discrete and process manufa...,#combine:0=252.5728702545166(discrete) #combin...
6976,6976,6976,how long did abraham lincoln serve,#combine:0=308.31992626190186(lincoln) #combin...
6977,6977,6977,does adult acne rosacea give you blepharitis,#combine:0=197.66818284988403(adult) #combine:...
6978,6978,6978,how long do you bake muffins,#combine:0=288.68935108184814(#base64(IyNmZmlu...


In [30]:
query_reps = splade.model(q_kwargs=splade.tokenizer(
                        "Now this is big long question how this thing will work now?",
                        add_special_tokens=True,
                        padding="longest",  # pad to max sequence length in batch
                        truncation="longest_first",  # truncates to max model length,
                        max_length=splade.max_length,
                        return_attention_mask=True,
                        return_tensors="pt",
                    ).to(splade.device))["q_rep"]

In [31]:
query_reps.shape

torch.Size([1, 30522])

In [32]:
import torch

cols = torch.nonzero(query_reps[0]).squeeze().cpu().tolist()
# and corresponding weights               
weights = query_reps[0,cols].cpu().tolist()

In [33]:
len(cols)

69

In [34]:
len(weights)

69

In [39]:
import numpy as np



In [43]:
#m = np.memmap('msmarco-passage-splade-pisa/fwd', mode='r', dtype=np.uint32)
lexicon = [l.strip() for l in open('msmarco-passage-splade-pisa/fwd.terms','rt')]

In [27]:
import torch
with torch.no_grad():
    doc_reps = splade.model(d_kwargs=splade.tokenizer(
                        doc,
                        add_special_tokens=True,
                        padding="longest",  # pad to max sequence length in batch
                        truncation="longest_first",  # truncates to max model length,
                        max_length=splade.max_length,
                        return_attention_mask=True,
                        return_tensors="pt",
                    ).to(splade.device))["d_rep"]
    
    # for per each question
    
    cols = torch.nonzero(doc_reps[0]).squeeze().cpu().tolist()
    # and corresponding weights               
    weights = doc_reps[0,cols].cpu().tolist()
    
    d = {splade.reverse_voc[k] : v for k, v in sorted(zip(cols, weights), key=lambda x: (-x[1], x[0]))}


In [30]:
d

{'iso': 2.170837879180908,
 'termination': 1.9991546869277954,
 'combine': 1.9783039093017578,
 'divide': 1.8066225051879883,
 '##pati': 1.598440170288086,
 '##balance': 1.5944828987121582,
 'merge': 1.5135806798934937,
 '##º': 1.4936060905456543,
 '##ssi': 1.4812493324279785,
 '##lating': 1.4776906967163086,
 'zero': 1.3979449272155762,
 '0': 1.3730231523513794,
 'di': 1.1965231895446777,
 'specify': 1.1530905961990356,
 '##r': 1.0828639268875122,
 'terminated': 1.020087480545044,
 'input': 1.0030416250228882,
 'combination': 0.9944084882736206,
 'power': 0.9754437208175659,
 'consider': 0.8965662121772766,
 'enable': 0.8780695199966431,
 'failure': 0.8625272512435913,
 'balance': 0.7969439625740051,
 'merger': 0.7960633039474487,
 'device': 0.7737906575202942,
 '##rs': 0.755608320236206,
 'generator': 0.7124901413917542,
 'fail': 0.7047978043556213,
 'failures': 0.6902132034301758,
 'brake': 0.6813592314720154,
 'end': 0.672426164150238,
 'ending': 0.6374238729476929,
 'un': 0.611801

In [2]:
dataset = pt.get_dataset('irds:msmarco-passage')

In [3]:
for i,doc in enumerate(dataset.get_corpus_iter()):
    print(i, doc)
    if i>10:
        break
    

msmarco-passage documents:   0%|          | 11/8841823 [00:00<05:54, 24952.59it/s]

0 {'text': 'The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.', 'docno': '0'}
1 {'text': 'The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science.', 'docno': '1'}
2 {'text': 'Essay on The Manhattan Project - The Manhattan Project The Manhattan Project was to see if making an atomic bomb possible. The success of this project would forever change the world forever making it known that something this powerful can be manmade.', 'docno': '2'}
3 {'text': 'The Manhattan Project was the name for a project conducted during World War II, to develop the first atomic bomb. It refers specifically to the period 